# Create Spotify playlist from list

In [79]:
import requests
import json
import re
import pandas as pd

title = "(?P<title>\w+)"
artist = "(?P<artist>\w+)"
holder = "(?P<word>\w+)"

# Generate track id list
def append_id(text):
    """extract and append track id to song_list
    """
    
    track = re.findall(r'"spotify:track:(.*?)"', text)[0]
    if track is not None:
        song_list.append(track)
    return song_list

def spotify_search(term):
    """search for songs on spotify using api and append_id to song_list
    """
    
    if isinstance(term, float):
        return False

    # api request to search
    url = "https://api.spotify.com/v1/search?q="+term+"&type=track"
    headers = {'Accept' : 'application/json', 'Content-Type' : 'application/json', 'Authorization' : 'Bearer ' + token}
    r = requests.get(url, headers=headers)
    parsed_json = json.loads(r.text)
    
    if parsed_json['tracks']['total'] != 0:
        append_id(r.text)
    else:
        # song unavailable
        return False
    return True


# Generate spotify list
def spotify_add(term):
    """add song to spotify playlist by track id
    """
    # api request to add
    url = "https://api.spotify.com/v1/playlists/"+playlist_id+"/tracks?uris=spotify%3Atrack%3A"+term
    headers = {'Accept' : 'application/json', 'Content-Type' : 'application/json', 'Authorization' : 'Bearer ' + token}
    r = requests.post(url, headers=headers)
    return



def define_pattern(pattern):
#     pattern = "%tracknumber% %artist% %%"

    var_names = re.findall(r'%(.*?)%', pattern)
    for var_name in var_names:
        pattern = re.sub("%" +var_name + "%", "(?P<" + var_name + r'>\\w+)', pattern)
        pattern = pattern.replace(r'(?P<>\w+)', r'(\w+)')

    print(pattern)
    return pattern


# Text processing
def remove_num(text):
    text = re.sub("(\d+)", "", text)
    return text

def remove_sym(text):
    text = re.sub(r'(\W)\1+', r' ', text)
    
    text = text.replace(".", " ").replace("，", "").replace("–", "").replace("*", "").replace(
        "-", " ").replace("·", " ").replace("’", " ").replace("/", " ")
#     text = text.replace("(", " ").replace("（", " ").replace(")", "").replace("）", "")
#     text = text.replace("<", " ").replace(">", "")
#     text = text.replace("《", " ").replace("》", " ")
#     text = text.replace("[", "").replace("]", "").replace("「", "").replace("」", "")
    
    return text

def remove_line(text):
    lines = [line.strip() for line in text.split('\n') if line.strip() != '']
    return lines
   
def split_line(line, pos1=0, pos2=-1):
    songs_to_search = line.split(" ")[pos1] + " " + line.split(" ")[pos2]
    return songs_to_search


def text_process(text, num=True):
    if not num:
        text = remove_num(text)
    text = remove_sym(text)
    
    text = text.replace("[粤]", "").replace("演唱：", "")
    text = text.replace("林隆旋", "林隆璇")
    text = text.replace("眞", "真")
    text = text.replace("\t", " ")
    #text = text.replace("  ", " ")
    lines = remove_line(text)

    return lines

def extract_text(lines, pattern):
    songs_to_search = lines
    
    for i in range(len(songs_to_search)):
        m = songs_to_search[i]
        
        try:
            m = re.match(pattern, m)
            songs_to_search[i] = ' '.join([m[k] for k in m.groupdict()])
        except:
            # the hardcoded way
            print("\n======hardcode======")
            print(songs_to_search[i])
            split_line(songs_to_search[i])
            print(songs_to_search[i])
            
    return songs_to_search


    
# Hard code to split lines
def split_text(lines, pos1=0, pos2=-1):
    songs_to_search = lines
    
    for i in range(len(songs_to_search)):
        songs_to_search[i] = split_line(songs_to_search[i], pos1, pos2)
            
    return songs_to_search


# Web data processing with pandas
def extract_web(link):
    website = pd.read_html(link)
    song_list = []

    all_songs = website[17]
    all_songs

    all_songs.rename(columns=cus_columns, inplace=True)

    if one:
        songs_to_search = all_songs[('十大中文金曲得獎名單（按頒獎次序排序）',  'title')]

    if two:
        all_songs['list1'] = all_songs["title1"] + ' ' +  all_songs["artist1"]
        all_songs['list2'] = all_songs["title2"] + ' ' +  all_songs["artist2"]

        songs_to_search = all_songs["list1"]
        songs_to_search = songs_to_search.append(all_songs["list2"])
        # songs_to_search.drop(index=25, inplace=True)
        
    return songs_to_search

# Extract multiple songs according to artists


# title_between = r'《(.*?)》'
def extract_composer(lines, artist_split="：", title_split="，"):
    lines = [line.strip() for line in text.split('\n') if line.strip() != '']
    songs_to_search = []
    
    for line in lines:
        artist = line.split(artist_split)[0]
        title_str = line.split(artist_split)[1]

        if len(title_split) == 1:
            titles = title_str.split(title_split)
        else:
            titles = re.findall(title_between, title_str)
            
        for title in titles:
            songs_to_search.append(artist + " " + title)
                
    return songs_to_search

# print(songs_to_search)

#### Request for API token.

https://developer.spotify.com/console/post-playlist-tracks/?playlist_id=&position=&uris=

In [29]:
# pattern = "(\w+) " +title+ "--" +artist+ ""
# string = text.splitlines()[0].replace("\t", " ")
string = "呢喃 黃鶯鶯)"
print("String: " + string)
# pattern = title+ " " +artist  # + " " +holder

m = re.match(pattern, string)
m.groupdict()
[m[k] for k in m.groupdict()]
string = ' '.join([m[k] for k in m.groupdict()])
print(string)
# print(m.groupdict().values())

String: 呢喃 黃鶯鶯)


AttributeError: 'NoneType' object has no attribute 'groupdict'

In [110]:
text = """张杰——第一张
陈耀川——我始终爱你，赤子心，等一个爱我的人，感情线，女人，我们分享每一天，心情真好，一个午睡时做的梦，爱太浓，挽回她还不如忘了吧，给你一份温柔，情到深处，共度一生，接受我的心，只要有你能懂我
刘德华——天意，忘情水，为何爱上你，相思成灾，友谊历久一样浓，真永远，中国人，同名男子，真的爱着你，别说爱情苦，谢谢你的爱
蔡幸娟——我心已许，我只在乎真心，相爱容易相处难，缘分，秋凉如我心，我不要别人的爱，生死永不分，从来没想过这样道别离
梅艳芳——空城，女儿红，女人花，我看着寂寞长大，一生爱你千百回
陈德容——聚散不由我
陈明——我要找到你
韩雪——想起
李玟——不变的诺言，爱你在每一天
何家劲——认认真真过日子
陈志朋——用尽一生的温存
飞碟群星——相亲相爱
叶倩文——我该微笑还是哭，我知我会后悔，失恋一次上了瘾
蔡淳佳——眼睛，有心人有情人
郭富城——爱上我不快乐吗，谁会记得我，散场，真的谢谢你
姜育恒——寂寞的门，历尽沧桑的温柔，一如往昔
黎姿——我只怨自己
李碧华——分手
林忆莲——走在大街的女子，让我笑吧
林志炫——单身情歌
陆毅——告白，壮志雄心
吕方——多爱你一天，老情歌
满文军——故事，今生今世，这一颗心
孟庭苇——风中有朵雨做的云
潘越云——七重门，你会爱我很久吗，吻上你的心，我所爱的你，你是欲留还走的风
王杰——谁能了解我的心，如何相信这是场梦，爱你依然深似海
温兆伦——天地情缘，别再停留
吴奇隆——你是我等了好久好久的那个人，谢谢你陪我，思念就像一条河
苏有朋——爱人的心是不是在乎我，我知道我在做什么
吴宗宪——爱说笑的人
小虎队——不要难过，我想先离开，做我的新舞伴吧，为梦想找一个家，青春狂想曲，如梦初醒，燃烧青春火焰，意乱情迷，季节改变
谢霆锋——要我怎么忘了她
伊能静——红颜美人多薄命
张信哲——信仰
张惠妹——离爱出走
周迅——飘摇
叶欢——人生如梦，留话在风中，意乱情迷，再见谎言，几多深情几多愁
周慧敏——保护，承认爱，爱你多年 感情的分礼
钟汉良——在你身边
周华健——我愿意等待，不要在梦里想我
林慧萍——当初我怎么会不懂
孙楠——我们都是伤心人
李惠敏——爱上寂寞
红孩儿——答应我
群星——和未来有约，相亲相爱
叶蕴仪——诺言，是你答应说要对我好，我是谁的宝贝，把我放在心里头，好希望你也在这里，轻轻柔柔网住我，欲言又止
刘德华——相约到永久
蔡幸娟——再牵你的手
王子鸣VS韩雪——开心就好
马海生——谢谢你的爱2009，这一秒
张敬轩——吻得太逼真"""

playlist_id = '7DZV6oKBobOQVnYS8s89jD'
token = "BQAcg1bfiQotFYh4A6rKr6_I2vXPSCjqJGVoyrWpwWUdiMrewyGRTtWaaS-I-iunv-Ipa5B02jEfF94THYvAw3i-bgevgyJds5uxCyOvPcwQldmDjhNlpesjVR0tP_I-NA80Wn5cWzoe8grFZGw4tieRfMYqpSbWr6ubpX72IVD9HPZyKJS7avFBbvRHs9gYBSfS6Q"

# cus_columns = {"歌曲": "title", "主唱": "artist"}
# cus_columns = {"第一季金曲": "title1", "第二季金曲": "title2", "主唱": "artist1", "主唱.1": "artist2"}
link = None

pattern = "%title% %artist%"

# if text:
#     pattern = define_pattern(pattern)
#     text = re.sub(r'《(.*?)》', r'', text)
#     lines = text_process(text, num=False)
    
# #     songs_to_search = lines
# #     songs_to_search = extract_text(lines, pattern)
# #     songs_to_search = extract_composer(text)
#     songs_to_search = split_text(lines, 0, 1)
# elif link:
#     songs_to_search = web_process(link)

# songs_to_search = extract_composer(text, artist_split="——")

张


IndexError: list index out of range

In [34]:
songs_to_search

['张杰 第一张',
 '陈耀川 我始终爱你',
 '陈耀川 赤子心',
 '陈耀川 等一个爱我的人',
 '陈耀川 感情线',
 '陈耀川 女人',
 '陈耀川 我们分享每一天',
 '陈耀川 心情真好',
 '陈耀川 一个午睡时做的梦',
 '陈耀川 爱太浓',
 '陈耀川 挽回她还不如忘了吧',
 '陈耀川 给你一份温柔',
 '陈耀川 情到深处',
 '陈耀川 共度一生',
 '陈耀川 接受我的心',
 '陈耀川 只要有你能懂我',
 '刘德华 天意',
 '刘德华 忘情水',
 '刘德华 为何爱上你',
 '刘德华 相思成灾',
 '刘德华 友谊历久一样浓',
 '刘德华 真永远',
 '刘德华 中国人',
 '刘德华 同名男子',
 '刘德华 真的爱着你',
 '刘德华 别说爱情苦',
 '刘德华 谢谢你的爱',
 '蔡幸娟 我心已许',
 '蔡幸娟 我只在乎真心',
 '蔡幸娟 相爱容易相处难',
 '蔡幸娟 缘分',
 '蔡幸娟 秋凉如我心',
 '蔡幸娟 我不要别人的爱',
 '蔡幸娟 生死永不分',
 '蔡幸娟 从来没想过这样道别离',
 '梅艳芳 空城',
 '梅艳芳 女儿红',
 '梅艳芳 女人花',
 '梅艳芳 我看着寂寞长大',
 '梅艳芳 一生爱你千百回',
 '陈德容 聚散不由我',
 '陈明 我要找到你',
 '韩雪 想起',
 '李玟 不变的诺言',
 '李玟 爱你在每一天',
 '何家劲 认认真真过日子',
 '陈志朋 用尽一生的温存',
 '飞碟群星 相亲相爱',
 '叶倩文 我该微笑还是哭',
 '叶倩文 我知我会后悔',
 '叶倩文 失恋一次上了瘾',
 '蔡淳佳 眼睛',
 '蔡淳佳 有心人有情人',
 '郭富城 爱上我不快乐吗',
 '郭富城 谁会记得我',
 '郭富城 散场',
 '郭富城 真的谢谢你',
 '姜育恒 寂寞的门',
 '姜育恒 历尽沧桑的温柔',
 '姜育恒 一如往昔',
 '黎姿 我只怨自己',
 '李碧华 分手',
 '林忆莲 走在大街的女子',
 '林忆莲 让我笑吧',
 '林志炫 单身情歌',
 '陆毅 告白',
 '陆毅 壮志雄心',
 '吕方 多爱你一天',
 '吕方 老情歌',
 '满文军 故事',
 '满文军 今生今世',
 '满文军 这一颗心',
 '孟庭苇 风中有朵雨做的云',
 

In [121]:
na = ""
song_list = []

for i in songs_to_search:
    if not spotify_search(i):
        print(" unavailable          ", end="")
        na = na + ", " + i
    print(i)
    
# create playlist and add
for i in song_list:
    spotify_add(i)
    
print("\n缺：" + na[2:])

黄莺莺 你在我灵魂的边缘
 unavailable          叶欢  有谁能明暸
 unavailable          合辑  不是不在乎你
 unavailable          忧欢派 如果一定要离去
王芷蕾 是否依然深爱着我
王杰  你是我胸口永远的痛
姜育恒 今夜星光灿烂
 unavailable          星星‧ 爱情电话
 unavailable          叶欢  多说一些说甜一些
 unavailable          合辑  说在我心中的爱
小虎队 你的眼睛下着雨
姜育恒 谁能忘了我
 unavailable          合辑 我的爱我的歌
 unavailable          小虎队 男孩不哭
叶欢  找个地方偷偷想你
 unavailable          姚黛玮 危险的吸引力
 unavailable          李之勤 如果你的心里有两个人
 unavailable          小虎队 最爱你哭泣时候的眼睛
王杰  亲爱的你是我最后的爱
 unavailable          陈秀男 弹琴说爱
邰正宵 最爱你哭泣时候的眼睛
小虎队 唱我最爱的歌给你听
蔡幸娟 不到天荒情不老
郭富城 说你爱我
 unavailable          金玉岚 金色阳光
 unavailable          方志耀 我的心
 unavailable          林忆莲 哈啰寂寞
 unavailable          方志耀 因为有你
郭富城 我是不是该安静的走开
 unavailable          蔡幸娟 用多少伤心能换回你
王杰  流浪的心
马萃如 什么样的爱你才会懂
郭富城 到底有谁能 够告诉我
 unavailable          叶倩文 潇洒走一回
小虎队 放心去飞
 unavailable          少女队 我的心要去旅行
 unavailable          郭小霖 留不住
林志颖 不是每个恋曲都有美好回忆
 unavailable          郭富城 请把我的情感带回家
 unavailable          红孩儿 你最好永远别爱上我
 unavailable          叶蕴仪 欺骗我的心
林子祥 选择
伊能静 

### Custom 

In [118]:
text = """1987.8 黄莺莺《雪在烧》专辑  《你在我灵魂的边缘》 作曲：陈秀男
1987.9 叶欢《因为爱你》专辑  《有谁能明暸》作曲：陈秀男  

1988.2 姜育恒《一世情缘》专辑  制作：陈秀男 姜育恒
1988.4 叶欢《放你的真心在我的手心》专辑  制作：陈秀男
1988.5 合辑《六个朋友》专辑        《不是不在乎你》 作曲：陈秀男
1988.6 忧欢派对《告别17，微笑18》专辑    《如果一定要离去》作曲：陈秀男
1988.6 王芷蕾《悲欢岁月》专辑     《是否依然深爱着我》作曲：陈秀男
1988.7 王杰《忘了你 忘了我》专辑    《你是我胸口永远的痛》作曲：陈秀男
1988.9 姜育恒《跟往事干杯》专辑   制作：陈秀男《今夜星光灿烂》，《每一个辗转不眠的夜》作曲：陈秀男
1988.9 星星‧月亮‧太阳《我有自己的路走》专辑  制作：陈秀男《爱情电话》，《心愿》作曲：陈秀男
1988.10 叶欢《记得我们有约》专辑 制作：陈秀男《多说一些说甜一些》，《不眠的夜》作曲：陈秀男

1989.1 忧欢派对/小虎队《新年快乐》专辑   制作：陈秀男       《新年快乐》 ，《你是我幼稚的情人》作曲：陈秀男
1989.3 合辑《七匹狼》专辑  制作：陈秀男 翁孝良《说在我心中的爱》，《没有》，《永远的蓝天》作曲：陈秀男
1989.4 小虎队《逍遥游》专辑  制作：陈秀男《你的眼睛下着雨》，《我的爱正年轻》，《探险风筝》，《星星宝贝》作曲：陈秀男
1989.6 姜育恒《多年以后·再回首》专辑  制作：陈秀男 姜育恒   《谁能忘了我》作曲：陈秀男
1989.8 王杰《孤星》专辑  制作：陈秀男
1989.8 合辑《1989夏令营》专辑《我的爱我的歌》，《初恋的滋味》作曲：陈秀男
1989.9 小虎队《男孩不哭》专辑 《男孩不哭》作曲：陈秀男
1989.11 叶欢《谁在秋天捡到我的心》专辑 制作：陈秀男《找个地方偷偷想你》，《你的爱烧痛我的心》，《不要皱着眉头看我》作曲：陈秀男
1989.12 姚黛玮《危险的吸引力》专辑     《危险的吸引力》作曲：陈秀男

1990.1 红孩儿《亮出我的年轻护照》专辑 《何去何从》 作曲：陈秀男
1990.1 李之勤《长痛还是短痛》专辑 《如果你的心里有两个人》作曲：陈秀男
1990.1 王杰《向太阳怒吼》专辑  制作：陈秀男 王杰 Ricky Ho
1990.2 小虎队《红蜻蜓》专辑《最爱你哭泣时候的眼睛》，《为了你我会做到》，《骊歌》作曲：陈秀男
1990.5 王杰《我要飞》专辑 《亲爱的你是我最后的爱》作曲：陈秀男
1990.7 陈秀男《弹琴说爱》钢琴演奏专辑系列，一套5张
1990.8 邰正宵《痴心的我》专辑  制作：陈秀男 丁晓雯 《最爱你哭泣时候的眼睛》（翻唱小虎队作品），《阳光住在我心里》作曲：陈秀男
1990.9 小虎队《星星的约会》专辑 《唱我最爱的歌给你听》作曲：陈秀男
1990.9 蔡幸娟《是雨还是你》专辑     《不到天荒情不老》作曲：陈秀男
1990.9 郭富城《对你爱不完》专辑  制作：陈秀男   《说你爱我》，《蓝色眼睛的爱情》作曲：陈秀男
1990.10 金玉岚《迷路的天使》专辑          《金色阳光》作曲：陈秀男
1990.10 方志耀《我是个无心的人》专辑          《我的心》作曲：陈秀男
1990.12 林忆莲《爱上一个不回家的人》专辑 制作：陈秀男 许愿 Dick Lee Makoto Kubota《哈啰寂寞》，《爱伤害了你我》作曲：陈秀男

1991.1 王杰《为了爱 梦一生》专辑 《如果你真的在乎我》 作曲：陈秀男
1991.3 方志耀《留一些空间》专辑 《因为有你》作曲：陈秀男
1991.4 郭富城《我是不是该安静的走开》专辑  制作：陈秀男《我是不是该安静的走开》作曲：陈秀男
1991.4 蔡幸娟《真的让我爱你吗》专辑 《用多少伤心能换回你》作曲：陈秀男
1991.7 王杰《忘记你不如忘记自己》专辑   《流浪的心》作曲：陈秀男
1991.7 林子祥《这次你是真的伤了我的心》专辑  制作：陈秀男 林子祥
1991.8 马萃如《什么样的爱你才会懂》专辑  制作：陈秀男 丁晓雯 《什么样的爱你才会懂》作曲：陈大力/陈秀男，《爱我不爱》作曲：陈秀男
1991.10 郭富城《到底有谁能 够告诉我》专辑  制作：陈秀男《到底有谁能 够告诉我》作曲：陈秀男
1991.10 叶倩文《潇洒走一回》专辑  制作：陈秀男《潇洒走一回》，《我们的爱这么难》作曲：陈大力/陈秀男
1991.11 林忆莲《都市心》专辑  制作：陈秀男 许愿
1991.12 小虎队《再见》专辑  制作：陈秀男 李子恒   《放心去飞》作曲：陈大力/陈秀男，《骊歌（说好这次不掉眼泪）》作曲：陈秀男
1991.12 少女队《我的心要去旅行》专辑     《我的心要去旅行》作曲：陈秀男

1992.1 小虎队《Best Dance Remix》精选专辑  制作：陈大力 陈秀男 李子恒《男孩不哭》，《骊歌II（Farewell Version） 》 作曲：陈秀男
1992.2 郭小霖《相识不相恋》专辑 《留不住》作曲：陈秀男
1992.2 林志颖《不是每个恋曲都有美好回忆》专辑  制作：陈秀男 夏春湧《不是每个恋曲都有美好回忆》作曲：陈大力/陈秀男
1992.2 郭富城《请把我的情感带回家》精选专辑  制作：陈秀男
1992.3 红孩儿《故事》专辑   《你最好永远别爱上我》作曲：陈大力/陈秀男，《王者之师》作曲：陈秀男
1992.4 叶蕴仪《欺骗我的心》专辑  制作：陈秀男 陈耀川 蒋三省 熊美玲        《欺骗我的心》作曲：陈大力/陈秀男
1992.4 林子祥《这样爱过你》专辑  制作：陈秀男 《选择》作曲：陈大力/陈秀男
1992.5 伊能静《游戏》专辑  制作：陈秀男《我不过是你的游戏吗？》，《萤火虫》作曲：陈大力/陈秀男
1992.5 林立洋《负心》专辑        《游子吟》作曲：陈大力/陈秀男
1992.6 合辑《想飞就飞》专辑  制作：陈秀男 李子恒   《想飞就飞》，《旧情人的眼泪》作曲：陈大力/陈秀男
1992.6 林志颖《今年夏天》专辑     《今年夏天》作曲：陈大力/陈秀男

1992.7 王杰《英雄泪》专辑 《黑暗中的哭泣》作曲：陈秀男《希望》，《谁能够爱我到天长地久》作曲：陈大力/陈秀男
1992.7 叶倩文《真心真意过一生》专辑  制作：陈秀男 Tommy Page《真心真意过一生》，《曾经心疼》，《因为寂寞》作曲：陈大力/陈秀男
1992.7 郭富城《爱你》专辑  制作：陈秀男    《爱你》，《不要让你知道》作曲：陈大力/陈秀男，《永远我都会记得》作曲：陈秀男
1992.9 李之勤《想你的夜》专辑       《不该爱的是你》作曲：陈大力/陈秀男
1992.10 叶蕴仪《爱的故事》专辑  制作：陈秀男   《爱的故事》，《是你》，《蓝色爱琴海》作曲：陈大力/陈秀男
1992.11 马萃如《从来没有想过爱上你》专辑  制作：陈秀男   《全心演好每个自己》作曲：陈大力/陈秀男
1992.11 潘越云《心的诱惑》专辑     《来生相守》作曲：陈大力/陈秀男
1992.12 张雨生《大海》专辑          《大海》作曲：陈大力/陈秀男
1992.12 林志颖《为什么受伤的总是我》专辑                   《为什么受伤的总是我》，《我对天空说爱你》，《用我的真心宝贝你的心》作曲：陈大         力/陈秀男
1992.12 苏有朋《我只要你爱我》专辑             《勇气》作曲：陈大力/陈秀男

1993.1 林子祥《决定》专辑  制作：陈秀男 林子祥 《决定》，《重逢》作曲：陈大力/陈秀男
1993.1 王杰《我》专辑  《我》，《无悔无憾》，《回家》作曲：陈大力/陈秀男
1993.1 郭富城《把所有的爱都留给你》专辑  制作：陈秀男     《把所有的爱都留给你》，《情非得已离开你》，《爱到最后》作曲：陈大力/陈秀男
1993.2 蔡幸娟《半点东方心》专辑       《潇洒走一回》作曲：陈大力/陈秀男
1993.3 吴奇隆《追梦》专辑 《追梦》作曲：陈大力/陈秀男
1993.3 王韵禅《明天》专辑 《第一次》作曲：陈大力/陈秀男
1993.3 刘德华《真情难收》专辑  制作：陈秀男 杜自持    《一辈子的错》，《情歌》，《真情难收》作曲：陈大力/陈秀男
1993.4 温兆伦《永远是你》专辑  制作：陈秀男      《永远是你》，《寂寞的人》，《为我守候》作曲：陈大力/陈秀男
1993.4 伊能静《恋爱中的女人》专辑  制作：陈秀男     《天变地变情不变》，《轰轰烈烈去爱》作曲：陈大力/陈秀男，《谁？》作曲：陈秀男
1993.4 叶倩文《明月心》专辑  制作：陈秀男                  《永远》，《93‘潇洒走一回》，         《明月心》，《谁的心能让我停泊》作曲：陈大力/陈秀男
1993.7 苏有朋《等到那一天》专辑 《回到从前》作曲：陈大力/陈秀男
1993.7 林志颖《牵挂你的我》专辑   制作：陈秀男 蔡宗政 夏春湧    《懵懂少年心》，《全新的爱》，《就是这一天》，《戏梦》作曲：陈大力/陈秀男


1993.7 刘德华《我爱你》EP专辑  制作：陈秀男 杜自持   《硬汉子，热女人》作曲：陈大力/陈秀男（《真情难收》粤语版）
1993.8 林子祥《精选之天长地久》专辑  制作：陈秀男 林子祥    《天长地久》，《重逢》，《决定》，《选择》作曲：陈大力/陈秀男
1993.9 刘德华《一生一次》专辑制作：陈秀男 杜自持《一生一次》，《不用人陪》作曲：陈大力/陈秀
1993.10 郭富城《梦难留》专辑  制作：陈秀男《梦难留》，《爱你一生够不够》，《多情总是痛》作曲：陈大力/陈秀男
1993.10 吴奇隆《蓦然回首》专辑 《蓦然回首》，《死心不息》作曲：陈大力/陈秀男
1993.10 王杰《浪子心》精选专辑    《你是我胸口永远的痛》作曲：陈秀男  《回家》，《希望》作曲：陈大力/陈秀男
1993.10 叶欢《鸳鸯锦》精选专辑  制作：吴大卫 陈秀男 李寿全 楼文中《你是我胸口永远的痛》作曲：陈秀男 
1993.12 林子祥《When A Man Loves A Woman》专辑  制作：陈秀男 林子祥
1993.12 小虎队《星光依旧灿烂》专辑 《永远》（翻唱叶倩文作品）作曲：陈大力/陈秀男
1993.12 周葆元《Sorry seems to be the hardest word》专辑  制作：陈秀男      《爱在飘雪的午夜》作曲：陈秀男

1994.1 林志颖《火热的心》专辑 《火热的心》作曲：陈大力/陈秀男
1994.1 张雨生《自由歌》精选专辑   《大海》作曲：陈大力/陈秀男
1994.1 合辑《For Lovers Only》专辑《天长地久》，《永远》，《决定》，《爱的故事》，《我对天空说爱你》，《第一次》作曲：陈大力/陈秀男
1994.1 姜育恒《不朽金曲精选II》专辑   《今夜星光灿烂》作曲：陈秀男
1994.1 郭富城《一生的情人》精选专辑  制作：陈秀男      《把所有的爱都留给你》，《不要让你知道》，《爱你》，《爱到最后》，《梦难留》，《情非得已离开你》作曲：陈大力/陈秀男  《到底有谁能够告诉我》，《我是不是该安静的走开》作曲：陈秀男 
1994.1 叶倩文《离开情人的日子》专辑  制作：陈秀男《人生其实很简单》，《朋友》作曲：陈大力/陈秀男
1994.2 郭富城《梦的尽头就是天涯》专辑 制作：陈秀男  《天涯》，《我的心开始有落寞时候》，《爱你太深》作曲：陈大力/陈秀男  《深情的男人》作曲：陈秀男 
1994.4 吴奇隆《想找一个地方》专辑 《My Summer Dream》作曲：陈大力/陈秀男
1994.5 卓玉《中国少年》专辑 《中国少年》作曲：陈大力/陈秀男

1994.5 王杰《孤星梦》精选专辑 《我》作曲：陈大力/陈秀男
1994.5 李碧华《泪痕心事》专辑 制作：陈秀男     《我的世界》作曲：陈大力/陈秀男
1994.6 郭富城《渴望》专辑  制作：陈秀男  《渴望》，《说声我爱你》作曲：陈大力/陈秀男
1994.7 王杰《候鸟》专辑   《寂寞的人》作曲：陈大力/陈秀男
1994.8 周葆元《总有一天你会懂》专辑 制作：陈秀男《总有一天你会懂》作曲：陈秀男
1994.8 吕方《只想遇到一个人》专辑  制作：陈秀男
1994.9 吴奇隆《双飞》专辑 《飚》作曲：陈大力/陈秀男  
1994.11 叶倩文《潇洒明月心》精选专辑  制作：陈秀男《潇洒走一回》，《曾经心疼》，《明月心》，《选择》，《朋友》作曲：陈大力/陈秀         男
1994.11 叶蕴仪《把我放在心里头》精选专辑  制作：陈秀男  陈耀川 蒋三省《欺骗我的心》，《爱的故事》作曲：陈大力/陈秀男  

1995.1 郭富城《我的开始在这里》专辑  制作：陈秀男《狂恋》，《纠缠一辈子》作曲：陈秀男
1995.8 刘德华《登峰造极精华辑》精选专辑  《真情难收》，《一生一次》，《一辈子的错》作曲：陈大力/陈秀男
"""

In [113]:
# title_between = r'《(.*?)》'
def extract_composer(lines, artist_split="：", title_split="，"):
    songs_to_search = []
    
    for line in lines:
        artist = line.split(artist_split)[0][:3]
#         print(artist)
        title_str = line.split(artist_split)[1]

        if len(title_split) == 1:
            titles = title_str.split(title_split)
        else:
            titles = re.findall(title_between, artist_split + title_str)
#         print(titles)
            
        for title in titles:
            songs_to_search.append(artist + " " + title)
                
    return songs_to_search

In [119]:
text = re.sub("《(.*?)》专辑", "", text)
text = remove_num(text)
text = remove_sym(text)

lines = remove_line(text)
lines = [line for line in lines if "《" in line]

songs_to_search = extract_composer(lines, artist_split="《", title_split=r"《(.*?)》")

In [120]:
songs_to_search

['黄莺莺 你在我灵魂的边缘',
 '叶欢  有谁能明暸',
 '合辑  不是不在乎你',
 '忧欢派 如果一定要离去',
 '王芷蕾 是否依然深爱着我',
 '王杰  你是我胸口永远的痛',
 '姜育恒 今夜星光灿烂',
 '星星‧ 爱情电话',
 '叶欢  多说一些说甜一些',
 '合辑  说在我心中的爱',
 '小虎队 你的眼睛下着雨',
 '姜育恒 谁能忘了我',
 '合辑 我的爱我的歌',
 '小虎队 男孩不哭',
 '叶欢  找个地方偷偷想你',
 '姚黛玮 危险的吸引力',
 '李之勤 如果你的心里有两个人',
 '小虎队 最爱你哭泣时候的眼睛',
 '王杰  亲爱的你是我最后的爱',
 '陈秀男 弹琴说爱',
 '邰正宵 最爱你哭泣时候的眼睛',
 '小虎队 唱我最爱的歌给你听',
 '蔡幸娟 不到天荒情不老',
 '郭富城 说你爱我',
 '金玉岚 金色阳光',
 '方志耀 我的心',
 '林忆莲 哈啰寂寞',
 '方志耀 因为有你',
 '郭富城 我是不是该安静的走开',
 '蔡幸娟 用多少伤心能换回你',
 '王杰  流浪的心',
 '马萃如 什么样的爱你才会懂',
 '郭富城 到底有谁能 够告诉我',
 '叶倩文 潇洒走一回',
 '小虎队 放心去飞',
 '少女队 我的心要去旅行',
 '郭小霖 留不住',
 '林志颖 不是每个恋曲都有美好回忆',
 '郭富城 请把我的情感带回家',
 '红孩儿 你最好永远别爱上我',
 '叶蕴仪 欺骗我的心',
 '林子祥 选择',
 '伊能静 我不过是你的游戏吗？',
 '林立洋 游子吟',
 '合辑  想飞就飞',
 '林志颖 今年夏天',
 '叶倩文 真心真意过一生',
 '郭富城 爱你',
 '李之勤 不该爱的是你',
 '叶蕴仪 爱的故事',
 '马萃如 全心演好每个自己',
 '潘越云 来生相守',
 '张雨生 大海',
 '林志颖 为什么受伤的总是我',
 '苏有朋 勇气',
 '王杰  我',
 '郭富城 把所有的爱都留给你',
 '蔡幸娟 潇洒走一回',
 '吴奇隆 追梦',
 '王韵禅 第一次',
 '刘德华 一辈子的错',
 '温兆伦 永远是你',
 '伊能静 天变地变情不变',
 '叶倩文 永远',
 '苏有朋 回到从前',
 '林志颖 懵懂少年心',
 '林子祥 天长地久',
 

In [116]:
lines = [line for line in text.split('\n') if '专辑' not in line]

songs_to_search = []
artist = ""

for line in lines:
    if "《" not in line:
        artist = line
    else:
        match = re.search(r'《(.*?)》', line)
        songs_to_search.append(artist + " " + match.group(1))